# Benchmarking skforecast

This notebook benchmarks the performance (velocity) of the `skforecast` in its different versions and keeps track of the results.

**Notes**

+ In version `0.15.0` the binning of residuals was introduced in multi-series forecasters. This explains the increase in the time taken to fit the model.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

In [2]:
# !pip install --upgrade numpy < 2.0 && \
# pip install --upgrade pandas && \
# pip install --upgrade scikit-learn

In [3]:
# !pip install skforecast==0.15.1
# !pip install skforecast==0.14.0
# !pip install skforecast==0.13.0

In [4]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import sklearn
import joblib
from benchmarking import BenchmarkRunner, plot_benchmark_results
import skforecast
import platform
import psutil
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from skforecast.utils import *
if skforecast.__version__ >= '0.14.0':
    from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
    from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
    from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries
else:
    from skforecast.ForecasterAutoreg import ForecasterAutoreg
    from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
    from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
    from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
    from skforecast.model_selection import backtesting_forecaster, backtesting_forecaster_multiseries

In [5]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.9
skforecast version: 0.15.1
scikit-learn version: 1.6.1
pandas version: 2.2.3
numpy version: 2.2.5
Computer network name: joaquin-HP-ProBook-440-G6
Processor type: x86_64
Platform type: Linux-6.11.0-24-generic-x86_64-with-glibc2.39
Operating system: Linux
Operating system release: 6.11.0-24-generic
Operating system version: #24~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 25 20:14:34 UTC 2
Number of physical cores: 4
Number of logical cores: 8


In [6]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="'force_all_finite' was renamed to 'ensure_all_finite'"
)

# ForecasterRecursiveMultiSeries

In [7]:
# Mock data for benchmarking
# ==========================================================
n_series = 600
len_series = 2000
series_dict = {}
rng = np.random.default_rng(321)
for i in range(n_series):
    series_dict[f'series_{i}'] = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name=f'series_{i}'
    )
series_dataframe = pd.DataFrame(series_dict, index=series_dict['series_0'].index)

exog_dict = {}
rng = np.random.default_rng(321)
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=series_dict[k].index
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict[k] = exog

exog_dict_prediction = {}
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=pd.date_range(
                start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                periods=100,
                freq='h'
            )
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict_prediction[k] = exog

In [8]:
# Benchmarking speed ForecasterRecursiveMultiSeries
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
# regressor = LinearRegression()
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursiveMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )
else:
    forecaster = ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )

def ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict():
    forecaster.fit(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict():
    forecaster._create_train_X_y(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog():
    forecaster.fit(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog():
    forecaster._create_train_X_y(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict():
    forecaster._create_train_X_y(series=series_dataframe, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_single_series():
    _ = forecaster._create_train_X_y_single_series(
            y = series_dict['series_0'],
            exog = exog_dict['series_0'],
            ignore_exog = False,
        )

def ForecasterRecursiveMultiSeries_predict_exog_is_dict():
    forecaster.predict(steps=100, exog=exog_dict_prediction, suppress_warnings=True)

def ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict():
    _ = forecaster._create_predict_inputs(
            steps         = 100,
            exog         = exog_dict_prediction,
            check_inputs = True
        )

def ForecasterRecursiveMultiSeries__check_predict_inputs():
    check_predict_input(
        forecaster_name  = type(forecaster).__name__,
        steps            = 100,
        is_fitted        = forecaster.is_fitted,
        exog_in_         = forecaster.exog_in_,
        index_type_      = forecaster.index_type_,
        index_freq_      = forecaster.index_freq_,
        window_size      = forecaster.window_size,
        last_window      = pd.DataFrame(forecaster.last_window_),
        exog             = exog_dict_prediction,
        exog_type_in_    = forecaster.exog_type_in_,
        exog_names_in_   = forecaster.exog_names_in_,
        interval         = None,
        levels           = forecaster.series_names_in_,
        series_names_in_ = forecaster.series_names_in_,
        encoding         = forecaster.encoding
    )

def ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict_conformal():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                cv=cv,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_single_series, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog, forecaster=forecaster)

forecaster.fit(series=series_dict, exog=exog_dict)
runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__check_predict_inputs, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict_conformal, forecaster=forecaster)

Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_single_series with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries_predict_exog_is_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict with skforecast version 0.15.1
Benchmarking function: Fore

In [14]:
# Plot results
# ==============================================================================
display_df = False
selected_date = '2025-04-24'
selected_platform = None #'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df)
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

# ForecasterRecursive

In [10]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [11]:
# Benchmarking speed ForecasterRecursive
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursive(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )
else:
    forecaster = ForecasterAutoreg(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )

def ForecasterRecursive_fit():
    forecaster.fit(y=y, exog=exog)

def ForecasterRecursive__create_train_X_y():
    if skforecast.__version__ >= '0.14.0':
        forecaster._create_train_X_y(y=y, exog=exog)
    else:
        forecaster.create_train_X_y(y=y, exog=exog)


def ForecasterRecursive_predict():
    forecaster.predict(steps=100, exog=exog_prediction)

def ForecasterRecursive__create_predict_inputs():
    _ = forecaster._create_predict_inputs(
            steps        = 100,
            exog         = exog_prediction,
            check_inputs = True
        )
    
def ForecasterRecursive_backtesting():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )

def ForecasterRecursive_backtesting_conformal():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                interval=[5, 95],
                interval_method='conformal',
                cv=cv,
                metric='mean_squared_error',

                show_progress=False
            )
    else:
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=30, output_dir="./")
_ = runner.benchmark(ForecasterRecursive__create_train_X_y, forecaster=forecaster)

runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_fit, forecaster=forecaster)

runner = BenchmarkRunner(repeat=30, output_dir="./")
forecaster.fit(y=y, exog=exog)
_ = runner.benchmark(ForecasterRecursive_predict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive__create_predict_inputs, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_backtesting, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive_backtesting_conformal, forecaster=forecaster)

Benchmarking function: ForecasterRecursive__create_train_X_y with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_fit with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_predict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive__create_predict_inputs with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_backtesting with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_backtesting_conformal with skforecast version 0.15.1


In [12]:
# Plot results
# ==============================================================================
display_df = False
selected_date = '2025-04-24'
selected_platform = 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df)
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

# Summary of historical results

In [15]:
results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark_all['function_name'] = results_benchmark_all['function_name'].str.split('_', n=1).str[1]
results_benchmark_all = results_benchmark_all.groupby(['forecaster_name', 'skforecast_version', 'function_name'])['run_time_avg'].agg('median').reset_index()
results_benchmark_all = results_benchmark_all.sort_values(by=['function_name'])

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='MultiSeries Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='Single-Series Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
